In [2]:
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.utils import get_stop_words


In [3]:
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

import re
import string

In [4]:
summarizer = LexRankSummarizer()

In [84]:
path = '/Users/noahedelstein/Desktop/Parsons/design with ml/ml_final/ml_final_python/state-of-the-union/transcripts.json'

with open(path) as data_file:
  data = data_file.read()
  # print(data) 
  data_content = json.loads(data)
  president = [line['president'] for line in data_content]
  dates = [line['date'] for line in data_content]
#   print(date)
  transcripts = [line['transcript'] for line in data_content]
  # parser = [PlaintextParser.from_string(transcript, Tokenizer("english")) for line in data_content]
  
#   print(transcripts[0])

In [6]:
summarizer.stop_words = get_stop_words("english")

In [7]:
sum_lex = []
for transcript in transcripts:
    parser = PlaintextParser.from_string(transcript, Tokenizer("english"))
    summary = summarizer(parser.document,8)
    sum_lex.append(str(summary))
    # text = summary__repr__()

In [8]:
stop_words = set(stopwords.words('english'))

All the nations of the world—friend or foe—will find that America is strong America is proud and America is free.  Because only then can we truly make America great again.  It's been a long time since we had fair trade.  Should have never been in our country.  All the nations of the world—friend or foe—will find that America is strong America is proud and America is free.  Because only then can we truly make America great again.  It's been a long time since we had fair trade.  Should have never been in our country.


In [35]:
sum_lex = [w.replace("<Sentence:", "") for w in sum_lex]
sum_lex = [w.replace("(", "[") for w in sum_lex]
sum_lex = [w.replace(">", "") for w in sum_lex]
sum_lex = [w.replace(")", "]") for w in sum_lex]
sum_lex = [w.replace(",", "") for w in sum_lex]

In [88]:
print(sum_lex[0])

l


In [33]:
print(sum_lex)

Northwestern frontier have terminated.You will I am persuaded learn with no less concern than I communicate it that reiterated endeavors toward effecting a pacification have hitherto issued only in new and outrageous proofs of persevering hostility on the part of the tribes with whom we are in contest.  It will I presume be duly considered whether the occasion does not call for an exercise of liberality toward the families of the deceased.It must add to your concern to be informed that besides the continuation of hostile appearances among the tribes north of the Ohio some threatening symptoms have of late been revived among some of those south of it.A part of the Cherokees known by the name of Chickamaugas inhabiting five villages on the Tennessee River have long been in the practice of committing depredations on the neighboring settlements.It was hoped that the treaty of Holston made with the Cherokee Nation in July 1791 would have prevented a repetition of such depredations; but the 

In [68]:
def tokenize(text):
    # first change all to lower case
    text = text.lower()
    # remove the http urls with a regular expression
    remove_urls = re.sub(r"http\S+", "", text)
    # remove punctuation - small note - periods are not important but other punctuation
    # could be useful to leave in!
    remove_punctuation = remove_urls.translate(str.maketrans('','',string.punctuation))
    # tokenize our sentences into words
    tokens = nltk.word_tokenize(remove_punctuation)
    remove_stops = [w for w in tokens if not w in stop_words]
    stems = [PorterStemmer().stem(t) for t in remove_stops]
    return stems

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

# calculating the tf-idf may take a while
print("calculating tf-idf")
tfidf = TfidfVectorizer(tokenizer=tokenize)
tfs = tfidf.fit_transform(transcripts)

calculating tf-idf


In [70]:
from sklearn.decomposition import TruncatedSVD

print("reducing tf-idf to 2 dim")
tfs_reduced = TruncatedSVD(n_components=2, random_state=0).fit_transform(tfs)
print("done")

reducing tf-idf to 2 dim
done


In [71]:
print(tfs_reduced)

[[ 4.04167602e-01  3.58701260e-01]
 [ 4.60119675e-01  3.72034618e-01]
 [ 4.09314427e-01  4.36664316e-01]
 [ 4.42697662e-01  4.74247617e-01]
 [ 4.55825587e-01  5.01986131e-01]
 [ 4.78979969e-01  4.92122906e-01]
 [ 4.45095102e-01  4.64507266e-01]
 [ 4.60952834e-01  4.83752442e-01]
 [ 4.87272296e-01  4.87636515e-01]
 [ 4.85579141e-01  4.97330065e-01]
 [ 4.62702799e-01  4.25640812e-01]
 [ 4.43315228e-01  3.69291334e-01]
 [ 5.07086453e-01  4.51361366e-01]
 [ 4.81421158e-01  3.92114963e-01]
 [ 4.59603047e-01  4.33163198e-01]
 [ 4.09091149e-01  3.46554126e-01]
 [ 4.07127933e-01  3.68710018e-01]
 [ 4.81652279e-01  4.65850141e-01]
 [ 4.89700258e-01  5.27800543e-01]
 [ 5.04340523e-01  4.97579414e-01]
 [ 5.18171131e-01  5.23660146e-01]
 [ 4.95184220e-01  5.16845948e-01]
 [ 4.79576580e-01  4.83032826e-01]
 [ 5.35319223e-01  4.84478696e-01]
 [ 5.04148965e-01  4.95593265e-01]
 [ 5.44995803e-01  5.21197805e-01]
 [ 4.81227717e-01  4.70017209e-01]
 [ 4.92399189e-01  4.36807798e-01]
 [ 4.53870558e-01  4

In [72]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
scaled = ss.fit_transform(tfs_reduced)

In [73]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(scaled)

In [74]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=5, metric='cosine').fit(scaled)
_, closest = neighbors.kneighbors(kmeans.cluster_centers_)

In [75]:
for index, cluster_row in enumerate(closest):
    print('\n\nCluster number ' + str(index) + '\n\n')
    for col in cluster_row:
        print(dates[col], president[col])
        print('\n\n')



Cluster number 0


2014-01-28 Barack Obama



2011-01-25 Barack Obama



1988-01-25 Ronald Reagan



2001-02-27 George W. Bush



1992-01-28 George Bush





Cluster number 1


1883-12-04 Chester A. Arthur



1893-12-04 Grover Cleveland



1842-12-06 John Tyler



1839-12-02 Martin van Buren



1872-12-02 Ulysses S. Grant





Cluster number 2


1793-12-03 George Washington



1796-12-07 George Washington



1895-12-02 Grover Cleveland



1818-11-16 James Monroe



1819-12-07 James Monroe





Cluster number 3


1877-12-03 Rutherford B. Hayes



1869-12-06 Ulysses S. Grant



1903-12-07 Theodore Roosevelt



1888-12-03 Grover Cleveland



1830-12-06 Andrew Jackson





Cluster number 4


1959-01-09 Dwight D. Eisenhower



1950-01-04 Harry S. Truman



1974-01-30 Richard Nixon



1972-01-20 Richard Nixon



1948-01-07 Harry S. Truman





Cluster number 5


1965-01-04 Lyndon B. Johnson



1995-01-24 William J. Clinton



1968-01-17 Lyndon B. Johnson



1971-01-22 Richard Nixon



1978

In [87]:
print(sum_lex[0])

l


In [85]:
lookup = []

with open(path) as data_file:
    data = data_file.read()
    data_content = json.loads(data)
    transcripts = [line['transcript'] for line in data_content]
    dates = [line['date'] for line in data_content]
    presidents = [line['president'] for line in data_content]
    # text = [line[te=] for line in data_content]
    for date, president, cluster_pos, summed, speech in zip(dates, presidents, tfs_reduced, sum_lex, transcripts):
        lookup.append({
            "Date": date,
            "President": president,
            "Cluster_Pos": cluster_pos.tolist(),
            "Summmary": summed,
            "Speech": speech
        })

In [86]:
with open('sotu_tfidif_sum.json', 'w') as outfile:
    json.dump(lookup, outfile)